In [1]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Generate input data
np.random.seed(42)
n_samples = 1000
x = np.linspace(0, 10, n_samples)


# Create correlated noise with increasing magnitude
def scale_cov_matrix(x, base_cov, scale_factor=0.1):
    return base_cov * (1 + scale_factor * x)


cov_matrix = np.array([[0.25, 0.15, 0.1], [0.15, 0.49, 0.2], [0.1, 0.2, 0.36]])

correlated_noise = np.array(
    [
        np.random.multivariate_normal(
            mean=[0, 0, 0], cov=scale_cov_matrix(xi, cov_matrix)
        )
        for xi in x
    ]
)

# Generate three dependent variables with smooth, non-monotonic relationships and correlated noise
y1_ = 2 * np.sin(x) + 0.5 * x
y1 = y1_ + correlated_noise[:, 0]
y2_ = 3 * np.cos(0.5 * x) + 0.3 * x**2
y2 = y2_ + correlated_noise[:, 1]
y3_ = 1.5 * np.sin(0.7 * x) * np.cos(0.3 * x) + 0.2 * x
y3 = y3_ + correlated_noise[:, 2]
# Combine the data
data = np.column_stack((x, y1, y2, y3))
real_data = np.column_stack((x, y1_, y2_, y3_))

# Plot the relationships
fig, axs = plt.subplots(3, 1, figsize=(10, 15))
fig.suptitle("Relationships between input and outputs (with correlated noise)")

for i, ax in enumerate(axs):
    ax.scatter(x, data[:, i + 1], alpha=0.5)
    ax.set_xlabel("Input")
    ax.set_ylabel(f"Output {i+1}")

plt.tight_layout()
plt.show()

print("Data shape:", data.shape)
print("First few rows of the data:")
print(data[:5])

# Plot correlation matrix of the outputs
correlation_matrix = np.corrcoef(data[:, 1:].T)
plt.figure(figsize=(8, 6))
plt.imshow(correlation_matrix, cmap="coolwarm", vmin=-1, vmax=1)
plt.colorbar()
plt.title("Correlation Matrix of Outputs")
plt.xticks(range(3), ["y1", "y2", "y3"])
plt.yticks(range(3), ["y1", "y2", "y3"])
for i in range(3):
    for j in range(3):
        plt.text(j, i, f"{correlation_matrix[i, j]:.2f}", ha="center", va="center")
plt.tight_layout()
plt.show()

In [ ]:
import lightgbm as lgb
from lightgbmlss.model import LightGBMLSS
from lightgbmlss.utils import create_mv_dataset
from lightgbmlss.distributions.Gaussian import MultivariateGaussian

# Create lightgbm dataset
dtrain = create_mv_dataset(
    data[:, 0].reshape(-1, 1),
    data[:, 1:],
)

# Create lightgbm lss model and train
lgblss = LightGBMLSS(MultivariateGaussian(n_dim=3))
lgblss.train(params={"learning_rate": 0.1, "num_iterations": 10}, train_set=dtrain)

# Make predictions
preds = lgblss.predict(data[:, 0].reshape(-1, 1))
preds


Todo:

1. Get the loss values of:
   1. starting values
   2. predictions after 1-10 iterations
   3. The known starting values

In [ ]:
import torch
from torch.distributions.multivariate_normal import MultivariateNormal

dist = MultivariateNormal(
    loc=torch.tensor(preds.iloc[:, :3].to_numpy()),
    scale_tril=torch.tensor(preds.iloc[:, 3:].to_numpy().reshape(-1, 3, 3, order="C")),
)
loss = -torch.nansum(dist.log_prob(torch.tensor(data[:, 1:])))
loss


In [ ]:


dist = MultivariateNormal(
    loc=torch.tensor(real_data[:, 1:]),
    covariance_matrix=torch.tensor(np.tile(cov_matrix, (n_samples, 1, 1))),
)
loss = -torch.nansum(dist.log_prob(torch.tensor(data[:, 1:])))
loss

In [ ]:
def get_loss(params):

    # Create lightgbm lss model and train
    lgblss = LightGBMLSS(MultivariateGaussian(n_dim=3))
    lgblss.train(params=params, train_set=dtrain)
    preds = lgblss.predict(data[:, 0].reshape(-1, 1))
    dist = MultivariateNormal(
        loc=torch.tensor(preds.iloc[:, :3].to_numpy()),
        scale_tril=torch.tensor(preds.iloc[:, 3:].to_numpy().reshape(-1, 3, 3, order="C")),
    )
    loss = -torch.nansum(dist.log_prob(torch.tensor(data[:, 1:])))
    return loss
print(get_loss({"learning_rate": 1e-10, "num_iterations": 1}))
print(get_loss({"learning_rate": 0.01, "num_iterations": 10}))
print(get_loss({"learning_rate": 0.01, "num_iterations": 20}))
print(get_loss({"learning_rate": 0.01, "num_iterations": 50}))
print(get_loss({"learning_rate": 0.01, "num_iterations": 100}))
print(get_loss({"learning_rate": 0.1, "num_iterations": 1000}))



In [ ]:
data[:, 1:]

In [ ]:
fig, axes = plt.subplots(3, 1, figsize=(12, 18), sharex=True)
fig.suptitle("LightGBMLSS MV Gaussian Predictions", fontsize=16)

# Compute covariance matrices
st = preds.iloc[:, 3:].to_numpy().reshape(-1, 3, 3, order="C")
cov = np.zeros_like(st)
for i in range(st.shape[0]):
    cov[i] = st[i] @ st[i].T

for i in range(3):
    ax = axes[i]
    
    mean = preds[f"loc_{i}"]
    std = np.sqrt(cov[:, i, i])
    lower = mean - 1.96 * std
    upper = mean + 1.96 * std
    
    ax.plot(data[:, [0]], mean, label="Mean")
    ax.fill_between(data[:, 0], lower, upper, alpha=0.3, label="95% CI")
    ax.scatter(data[:, 0], data[:, i+1], alpha=0.5, label="Data")
    ax.set_ylabel(f"Y{i+1}")
    ax.legend()
    ax.grid(True)

axes[-1].set_xlabel("Input")
plt.tight_layout()
plt.show()

In [ ]:
preds